In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install squad_ru_bert

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset, load_metric
import json

### RuBERT for question answering.


In [ ]:
corpus = load_dataset('sberquad')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

In [ ]:
corpus_train = corpus['train'][::]

In [ ]:
with open("train.json", "w") as outfile:
    json.dump(corpus_train, outfile, indent=4, sort_keys=False)

In [ ]:
import pandas as pd
data = pd.read_json("train.json")

In [ ]:
data = data[['title', 'context', 'question', 'answers']]

In [ ]:
with open("train_new.json", "w") as outfile:
    json.dump(data.to_json(orient="records"), outfile, indent=4, sort_keys=False)

In [ ]:
# from deeppavlov import build_model, configs, train_model
# # model_ru = build_model(configs.squad.squad_ru_bert, download=True)

### Загрузка конфигурационного файла

In [ ]:
from deeppavlov import train_model, build_model
from deeppavlov.core.commands.utils import parse_config


# Конфигурационный файл для нашей модели
model_config = parse_config('squad_bert')

#  dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

~/.deeppavlov/downloads/squad/


### Поменяем пути загрузки и сохранения модели

In [ ]:
model_config['chainer']['pipe'][3]['load_path'] = '/content/drive/MyDrive/Colab Notebooks/model.pth.tar'
model_config['chainer']['pipe'][3]['save_path'] = '/content/drive/MyDrive/Colab Notebooks/model.pth.tar'

In [ ]:
model_config['chainer']

{'in': ['context_raw', 'question_raw'],
 'in_y': ['ans_raw', 'ans_raw_start'],
 'pipe': [{'class_name': 'torch_squad_transformers_preprocessor',
   'vocab_file': 'bert-base-cased',
   'do_lower_case': False,
   'max_seq_length': 384,
   'in': ['question_raw', 'context_raw'],
   'out': ['bert_features', 'subtokens', 'split_context']},
  {'class_name': 'squad_bert_mapping',
   'do_lower_case': False,
   'in': ['split_context', 'bert_features', 'subtokens'],
   'out': ['subtok2chars', 'char2subtoks']},
  {'class_name': 'squad_bert_ans_preprocessor',
   'do_lower_case': False,
   'in': ['ans_raw', 'ans_raw_start', 'char2subtoks'],
   'out': ['ans', 'ans_start', 'ans_end']},
  {'class_name': 'torch_transformers_squad',
   'pretrained_bert': 'bert-base-cased',
   'save_path': '/content/drive/MyDrive/Colab Notebooks/model.pth.tar',
   'load_path': '/content/drive/MyDrive/Colab Notebooks/model.pth.tar',
   'optimizer': 'AdamW',
   'optimizer_parameters': {'lr': 2e-05,
    'weight_decay': 0.01,

In [ ]:
model_config['train']

{'show_examples': False,
 'evaluation_targets': ['valid'],
 'log_every_n_batches': 250,
 'val_every_n_batches': 500,
 'batch_size': 10,
 'pytest_max_batches': 2,
 'pytest_batch_size': 5,
 'validation_patience': 10,
 'metrics': [{'name': 'squad_v1_f1', 'inputs': ['ans', 'ans_predicted']},
  {'name': 'squad_v1_em', 'inputs': ['ans', 'ans_predicted']},
  {'name': 'squad_v2_f1', 'inputs': ['ans', 'ans_predicted']},
  {'name': 'squad_v2_em', 'inputs': ['ans', 'ans_predicted']}],
 'class_name': 'torch_trainer'}

In [ ]:
model_config['dataset_reader']

{'class_name': 'squad_dataset_reader',
 'data_path': '~/.deeppavlov/downloads/squad/'}

In [ ]:
model_config['dataset_iterator']

{'class_name': 'squad_iterator', 'seed': 1337, 'shuffle': True}

In [ ]:
model = build_model(model_config, install=True, download=True)

2023-02-28 14:12:31.392 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz to /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz to /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz
100%|██████████| 1.18G/1.18G [00:53<00:00, 22.0MB/s]
2023-02-28 14:13:25.649 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz archive into /root/.deeppavlov/models/squad_torch_bert/cased/bert-base-cased
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/models/squad_torch_bert/cased/squad_torch_bert_cased.tar.gz archive into /root/.deeppavlov/models/squad_torch_bert/cased/bert-base-cased


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
# model.save()

2023-02-28 12:09:48.713 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 210: Saving model to /content/drive/MyDrive/Colab Notebooks/model.pth.pth.tar.
INFO:deeppavlov.core.models.torch_model:Saving model to /content/drive/MyDrive/Colab Notebooks/model.pth.pth.tar.


In [ ]:
# provide a path to the train file
model_config['dataset_reader']['data_path'] = '/contents/train_new.json'

In [ ]:
model = train_model(model_config, install=True, download=True)

2023-02-28 14:14:16.384 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/squad/squad_torch_bert_cased.tar.gz download because of matching hashes
2023-02-28 14:14:16.407 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/datasets/squad-v1.1.tar.gz to /contents/train_new.json/squad-v1.1.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/datasets/squad-v1.1.tar.gz to /contents/train_new.json/squad-v1.1.tar.gz
100%|██████████| 9.04M/9.04M [00:01<00:00, 8.75MB/s]
2023-02-28 14:14:18.211 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /contents/train_new.json/squad-v1.1.tar.gz archive into /contents/train_new.json
INFO:deeppavlov.core.data.utils:Extracting /contents/train_new.json/squad-v1.1.tar.gz archive into /

{"valid": {"eval_examples_count": 10570, "metrics": {"squad_v1_f1": 6.9966, "squad_v1_em": 0.1907, "squad_v2_f1": 6.9568, "squad_v2_em": 0.1987}, "time_spent": "0:04:36", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


INFO:train_report:{"valid": {"eval_examples_count": 10570, "metrics": {"squad_v1_f1": 6.9966, "squad_v1_em": 0.1907, "squad_v2_f1": 6.9568, "squad_v2_em": 0.1987}, "time_spent": "0:04:36", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
249it [03:10,  1.31it/s]
0it [00:00, ?it/s]
1it [00:00,  3.25it/s]

{"train": {"eval_examples_count": 10, "metrics": {"squad_v1_f1": 98.5714, "squad_v1_em": 90.0, "squad_v2_f1": 98.5714, "squad_v2_em": 90.0}, "time_spent": "0:07:48", "epochs_done": 0, "batches_seen": 250, "train_examples_seen": 2500, "loss": 2.967320939064026}}



INFO:train_report:{"train": {"eval_examples_count": 10, "metrics": {"squad_v1_f1": 98.5714, "squad_v1_em": 90.0, "squad_v2_f1": 98.5714, "squad_v2_em": 90.0}, "time_spent": "0:07:48", "epochs_done": 0, "batches_seen": 250, "train_examples_seen": 2500, "loss": 2.967320939064026}}
499it [06:22,  1.31it/s]
0it [00:00, ?it/s]
1it [00:00,  3.28it/s]

...


INFO:train_report:{"train": {"eval_examples_count": 10, "metrics": {"squad_v1_f1": 92.2222, "squad_v1_em": 77.7778, "squad_v2_f1": 83.0, "squad_v2_em": 70.0}, "time_spent": "5:56:52", "epochs_done": 1, "batches_seen": 15500, "train_examples_seen": 154999, "loss": 0.767860402226448}}

0it [00:00, ?it/s]
1it [00:00,  3.80it/s]
2it [00:00,  3.73it/s]
3it [00:00,  3.70it/s]
4it [00:01,  3.64it/s]
5it [00:01,  3.66it/s]
6it [00:01,  3.64it/s]
7it [00:01,  3.69it/s]
8it [00:02,  3.68it/s]
9it [00:02,  3.65it/s]
10it [00:02,  3.67it/s]
11it [00:02,  3.70it/s]
12it [00:03,  3.72it/s]
13it [00:03,  3.71it/s]
14it [00:03,  3.71it/s]
15it [00:04,  3.70it/s]
16it [00:04,  3.69it/s]
17it [00:04,  3.71it/s]
18it [00:04,  3.67it/s]
19it [00:05,  3.64it/s]
20it [00:05,  3.65it/s]
21it [00:05,  3.62it/s]
22it [00:06,  3.59it/s]
23it [00:06,  3.56it/s]
24it [00:06,  3.54it/s]
25it [00:06,  3.53it/s]
26it [00:07,  2.69it/s]
27it [00:08,  2.30it/s]
28it [00:08,  2.57it/s]
29it [00:08,  2.78it/s]
30it [00

{"valid": {"eval_examples_count": 10570, "metrics": {"squad_v1_f1": 87.8968, "squad_v1_em": 80.3966, "squad_v2_f1": 87.2452, "squad_v2_em": 79.7919}, "time_spent": "6:01:59", "epochs_done": 1, "batches_seen": 15500, "train_examples_seen": 154999, "impatience": 1, "patience_limit": 10}}


INFO:train_report:{"valid": {"eval_examples_count": 10570, "metrics": {"squad_v1_f1": 87.8968, "squad_v1_em": 80.3966, "squad_v2_f1": 87.2452, "squad_v2_em": 79.7919}, "time_spent": "6:01:59", "epochs_done": 1, "batches_seen": 15500, "train_examples_seen": 154999, "impatience": 1, "patience_limit": 10}}
6989it [2:42:25,  1.27it/s]
0it [00:00, ?it/s]
1it [00:00,  2.31it/s]

{"train": {"eval_examples_count": 10, "metrics": {"squad_v1_f1": 61.3333, "squad_v1_em": 60.0, "squad_v2_f1": 61.3333, "squad_v2_em": 60.0}, "time_spent": "6:05:15", "epochs_done": 1, "batches_seen": 15750, "train_examples_seen": 157499, "loss": 0.6972185461521149}}



INFO:train_report:{"train": {"eval_examples_count": 10, "metrics": {"squad_v1_f1": 61.3333, "squad_v1_em": 60.0, "squad_v2_f1": 61.3333, "squad_v2_em": 60.0}, "time_spent": "6:05:15", "epochs_done": 1, "batches_seen": 15750, "train_examples_seen": 157499, "loss": 0.6972185461521149}}
7184it [2:44:59,  1.25it/s]

In [ ]:
from deeppavlov import evaluate_model

# model = evaluate_model(<config_path>, install=True, download=True)

In [ ]:
!cp /root/.deeppavlov/models/squad_torch_bert/cased/bert-base-cased/model.pth.tar /content/drive/MyDrive/

In [ ]:
from deeppavlov import configs

In [ ]:
from pprint import pprint
model_config = json.load(open(configs.classifiers.insults_kaggle_bert))

In [ ]:
pprint(model_config['metadata']['variables'])

{'DOWNLOADS_PATH': '{ROOT_PATH}/downloads',
 'MODELS_PATH': '{ROOT_PATH}/models',
 'MODEL_PATH': '{MODELS_PATH}/classifiers/insults_kaggle_torch_bert',
 'ROOT_PATH': '~/.deeppavlov',
 'TRANSFORMER': 'bert-base-uncased'}
